In [60]:
import pandas as pd
import numpy as np
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

sns.set_style("whitegrid")

In [63]:
data = pd.read_csv("training_data.csv")
df = pd.DataFrame(data)
df.drop_duplicates()
missing_model_year = df["Model Year"].isna().sum()
df = df.dropna(subset=["Model Year"]).sort_values(by="Date", ascending=True).reset_index(drop=True)
df["Model Year"] = df["Model Year"].astype(int)

Assuming we have a somewhat cleaned dataset (removing Model Year missing values), sort by year,...

In [64]:
df = df.drop(columns="Region", axis = 1)

## Data Wrangling

In [65]:
categorical_cols = ["Vehicle Category", "Fuel Type", "Fuel Technology", "Electric Mile Range"]
for col in categorical_cols:
    df[col] = df[col].astype("category")
    

In [66]:
df.dtypes

Date                                                    int64
Vehicle Category                                     category
GVWR Class                                             object
Fuel Type                                            category
Model Year                                              int64
Fuel Technology                                      category
Electric Mile Range                                  category
Number of Vehicles Registered at the Same Address      object
Vehicle Population                                      int64
dtype: object

In [67]:
df["Number of Vehicles Registered at the Same Address"].apply(lambda x: repr(x)).unique()


array(["'≥4'", "'1'", "'2'", "'3'", "'Unknown'"], dtype=object)

In [7]:
# import unicodedata

# ordinal_mapping = {'1': int(1), '2': int(2), '3': int(3), unicodedata.name("\u03C0"): int(4), "Unknown": int(-1)}

# df["Number of Vehicles Registered at the Same Address"] = df["Number of Vehicles Registered at the Same Address"].astype(str).map(ordinal_mapping)

In [68]:
df["Number of Vehicles Registered at the Same Address"] = df["Number of Vehicles Registered at the Same Address"].replace({'1': 1, '2': 2, '3': 3, "\u22654": 4, "Unknown": -1})

df["GVWR Class"] = df["GVWR Class"].replace({"Not Applicable": -1, "Unknown": -1})

In [69]:
df.isnull().sum()

Date                                                 0
Vehicle Category                                     0
GVWR Class                                           0
Fuel Type                                            0
Model Year                                           0
Fuel Technology                                      0
Electric Mile Range                                  0
Number of Vehicles Registered at the Same Address    0
Vehicle Population                                   0
dtype: int64

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40450 entries, 0 to 40449
Data columns (total 9 columns):
 #   Column                                             Non-Null Count  Dtype   
---  ------                                             --------------  -----   
 0   Date                                               40450 non-null  int64   
 1   Vehicle Category                                   40450 non-null  category
 2   GVWR Class                                         40450 non-null  object  
 3   Fuel Type                                          40450 non-null  category
 4   Model Year                                         40450 non-null  int64   
 5   Fuel Technology                                    40450 non-null  category
 6   Electric Mile Range                                40450 non-null  category
 7   Number of Vehicles Registered at the Same Address  40450 non-null  int64   
 8   Vehicle Population                                 40450 non-null  int64   
d

In [71]:
from sklearn.model_selection import train_test_split

# Separate the feature and the target
X = df.drop(columns= "Vehicle Population")
y = df["Vehicle Population"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build a pipeline of training

In [72]:
from sklearn.pipeline import Pipeline
from category_encoders.target_encoder import TargetEncoder
from xgboost import XGBRegressor

In [73]:
categorical_features = ["Vehicle Category", "Fuel Type", "Fuel Technology", "Electric Mile Range"]
estimators = [
    ('encoder', TargetEncoder(cols=categorical_features)),  # Encode categorical features only
    ('clf', XGBRegressor(random_state=42))
]

pipe = Pipeline(steps = estimators)

# Set up hyper parameter tuning 

In [74]:
# from skopt import BayesSearchCV
# from skopt.space import Real, Integer

# search_space = {
#     'clf__max_depth': Integer(3, 8),  # Keep tree depth moderate
#     'clf__learning_rate': Real(0.02, 0.2, prior='log-uniform'),  # Not too slow, not too fast
#     'clf__n_estimators': Integer(100, 500),  # Limit the number of trees to balance performance
#     'clf__subsample': Real(0.6, 0.9),  # Randomly sample data to prevent overfitting
#     'clf__colsample_bytree': Real(0.5, 0.8),  # Use only a subset of features per tree
#     'clf__colsample_bylevel': Real(0.5, 0.8),
#     'clf__colsample_bynode': Real(0.5, 0.8),
#     'clf__reg_alpha': Real(0.01, 2.0),  # Light L1 regularization
#     'clf__reg_lambda': Real(0.1, 5.0),  # Mild L2 regularization
#     'clf__gamma': Real(0.0, 5.0)  # Prevent excessive tree splits
# }

# opt = BayesSearchCV(
#     estimator=pipe,
#     search_spaces=search_space,
#     cv=5,  # More cross-validation folds for stability
#     n_iter=20,  # Optimize in 20 rounds (higher = better)
#     scoring='neg_root_mean_squared_error',  
#     random_state=42
# )

In [75]:
col = 'GVWR Class'
X_train[col] = X_train[col].astype("category").cat.codes
X_test[col] = X_test[col].astype("category").cat.codes

In [76]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import warnings
warnings.filterwarnings("ignore")

sns.set_style("whitegrid")

Assuming we have a somewhat cleaned dataset (removing Model Year missing values), sort by year,...

In [77]:
df1 = pd.read_csv("cleaned_data.csv", index_col=0)
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40450 entries, 0 to 40449
Data columns (total 10 columns):
 #   Column                                             Non-Null Count  Dtype 
---  ------                                             --------------  ----- 
 0   Date                                               40450 non-null  int64 
 1   Vehicle Category                                   40450 non-null  object
 2   GVWR Class                                         40450 non-null  object
 3   Fuel Type                                          40450 non-null  object
 4   Model Year                                         40450 non-null  int64 
 5   Fuel Technology                                    40450 non-null  object
 6   Electric Mile Range                                40450 non-null  object
 7   Number of Vehicles Registered at the Same Address  40450 non-null  object
 8   Region                                             40450 non-null  object
 9   Vehicle Population    

In [79]:
df1 = df1.drop(columns="Region", axis = 1)

KeyError: "['Region'] not found in axis"

## Data Wrangling

In [80]:
categorical_cols = ["Vehicle Category", "Fuel Type", "Fuel Technology", "Electric Mile Range"]
for col in categorical_cols:
    df1[col] = df1[col].astype("category")
    

In [82]:
df1.dtypes

Date                                                    int64
Vehicle Category                                     category
GVWR Class                                             object
Fuel Type                                            category
Model Year                                              int64
Fuel Technology                                      category
Electric Mile Range                                  category
Number of Vehicles Registered at the Same Address      object
Vehicle Population                                      int64
dtype: object

In [83]:
df1["Number of Vehicles Registered at the Same Address"].apply(lambda x: repr(x)).unique()


array(["'≥4'", "'1'", "'2'", "'3'", "'Unknown'"], dtype=object)

In [22]:
# import unicodedata

# ordinal_mapping = {'1': int(1), '2': int(2), '3': int(3), unicodedata.name("\u03C0"): int(4), "Unknown": int(-1)}

# df["Number of Vehicles Registered at the Same Address"] = df["Number of Vehicles Registered at the Same Address"].astype(str).map(ordinal_mapping)

In [84]:
df1["Number of Vehicles Registered at the Same Address"] = df["Number of Vehicles Registered at the Same Address"].replace({'1': 1, '2': 2, '3': 3, "\u22654": 4, "Unknown": -1})

df1["GVWR Class"] = df1["GVWR Class"].replace({"Not Applicable": -1, "Unknown": -1})

In [86]:
df1.isnull().sum()
df.isnull().sum()

Date                                                 0
Vehicle Category                                     0
GVWR Class                                           0
Fuel Type                                            0
Model Year                                           0
Fuel Technology                                      0
Electric Mile Range                                  0
Number of Vehicles Registered at the Same Address    0
Vehicle Population                                   0
dtype: int64

In [87]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40450 entries, 0 to 40449
Data columns (total 9 columns):
 #   Column                                             Non-Null Count  Dtype   
---  ------                                             --------------  -----   
 0   Date                                               40450 non-null  int64   
 1   Vehicle Category                                   40450 non-null  category
 2   GVWR Class                                         40450 non-null  object  
 3   Fuel Type                                          40450 non-null  category
 4   Model Year                                         40450 non-null  int64   
 5   Fuel Technology                                    40450 non-null  category
 6   Electric Mile Range                                40450 non-null  category
 7   Number of Vehicles Registered at the Same Address  40450 non-null  int64   
 8   Vehicle Population                                 40450 non-null  int64   
dtypes

In [88]:
from sklearn.model_selection import train_test_split

# Separate the feature and the target
X = df1.drop(columns= "Vehicle Population")
y = df1["Vehicle Population"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [89]:
col = 'GVWR Class'
X_train[col] = X_train[col].astype("category").cat.codes.astype("int64")
X_test[col] = X_test[col].astype("category").cat.codes.astype("int64")

# Build a pipeline of training

In [91]:
from sklearn.pipeline import Pipeline
from category_encoders.target_encoder import TargetEncoder
from xgboost import XGBRegressor

In [90]:
categorical_features = ["Vehicle Category", "Fuel Type", "Fuel Technology", "Electric Mile Range"]
# estimators = [
#     ('encoder', TargetEncoder(cols=categorical_features)),  # Encode categorical features only
#     ('clf', XGBRegressor(random_state=42))
# ]

# pipe = Pipeline(steps = estimators)

# Set up hyper parameter tuning 

In [30]:
# from skopt import BayesSearchCV
# from skopt.space import Real, Integer

# search_space = {
#     'clf__max_depth': Integer(3, 8),  # Keep tree depth moderate
#     'clf__learning_rate': Real(0.02, 0.2, prior='log-uniform'),  # Not too slow, not too fast
#     'clf__n_estimators': Integer(100, 500),  # Limit the number of trees to balance performance
#     'clf__subsample': Real(0.6, 0.9),  # Randomly sample data to prevent overfitting
#     'clf__colsample_bytree': Real(0.5, 0.8),  # Use only a subset of features per tree
#     'clf__colsample_bylevel': Real(0.5, 0.8),
#     'clf__colsample_bynode': Real(0.5, 0.8),
#     'clf__reg_alpha': Real(0.01, 2.0),  # Light L1 regularization
#     'clf__reg_lambda': Real(0.1, 5.0),  # Mild L2 regularization
#     'clf__gamma': Real(0.0, 5.0)  # Prevent excessive tree splits
# }

# opt = BayesSearchCV(
#     estimator=pipe,
#     search_spaces=search_space,
#     cv=5,  # More cross-validation folds for stability
#     n_iter=20,  # Optimize in 20 rounds (higher = better)
#     scoring='neg_root_mean_squared_error',  
#     random_state=42
# )

In [92]:
# from sklearn.pipeline import Pipeline
# from category_encoders.target_encoder import TargetEncoder

# def objective(trial):
#     params = {
#         'max_depth': trial.suggest_int('max_depth', 3, 8),
#         'learning_rate': trial.suggest_loguniform('learning_rate', 0.02, 0.2),
#         'n_estimators': trial.suggest_int('n_estimators', 100, 500),
#         'subsample': trial.suggest_uniform('subsample', 0.6, 0.9),
#         'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 0.8),
#         'colsample_bylevel': trial.suggest_uniform('colsample_bylevel', 0.5, 0.8),
#         'colsample_bynode': trial.suggest_uniform('colsample_bynode', 0.5, 0.8),
#         'reg_alpha': trial.suggest_loguniform('reg_alpha', 0.01, 2.0),
#         'reg_lambda': trial.suggest_loguniform('reg_lambda', 0.1, 5.0),
#         'gamma': trial.suggest_uniform('gamma', 0.0, 5.0),
#         'random_state': 42
#     }

#     # Build pipeline with categorical encoding
#     pipe = Pipeline([
#         ('encoder', TargetEncoder(cols=categorical_cols)),
#         ('clf', XGBRegressor(**params))
#     ])

#     pipe.fit(X_train, y_train)
#     y_pred = pipe.predict(X_test)
#     rmse = np.sqrt(np.mean((y_test - y_pred) ** 2))
#     return rmse


In [94]:
import optuna

def objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.02, 0.2),
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'subsample': trial.suggest_uniform('subsample', 0.6, 0.9),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 0.8),
        'colsample_bylevel': trial.suggest_uniform('colsample_bylevel', 0.5, 0.8),
        'colsample_bynode': trial.suggest_uniform('colsample_bynode', 0.5, 0.8),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 0.01, 2.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 0.1, 5.0),
        'gamma': trial.suggest_uniform('gamma', 0.0, 5.0),
        'random_state': 42
    }

    # Encode categorical features
    X_train_encoded = X_train.copy()
    X_test_encoded = X_test.copy()
    encoder = TargetEncoder(cols=categorical_cols)
    X_train_encoded[categorical_cols] = encoder.fit_transform(X_train[categorical_cols], y_train)
    X_test_encoded[categorical_cols] = encoder.transform(X_test[categorical_cols])

    model = XGBRegressor(**params)
    model.fit(X_train_encoded, y_train)
    y_pred = model.predict(X_test_encoded)
    rmse = np.sqrt(np.mean((y_test - y_pred) ** 2))
    return rmse

def print_rmse(study, trial):
    print(f"Trial {trial.number}: RMSE {trial.value}")

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=25, callbacks=[print_rmse]) # Can add callbacks=[print_rmse] to print RMSE here


[I 2025-02-01 16:04:43,187] A new study created in memory with name: no-name-814b69a2-2b5b-4560-a514-6f9187b74281
[I 2025-02-01 16:04:43,721] Trial 0 finished with value: 6647.616758420024 and parameters: {'max_depth': 7, 'learning_rate': 0.026435091969102858, 'n_estimators': 349, 'subsample': 0.6349706729659274, 'colsample_bytree': 0.7980728413941408, 'colsample_bylevel': 0.7367967734095607, 'colsample_bynode': 0.5011020640633209, 'reg_alpha': 0.012242804607734732, 'reg_lambda': 1.1175957431073547, 'gamma': 0.029314573809290922}. Best is trial 0 with value: 6647.616758420024.


Trial 0: RMSE 6647.616758420024


[I 2025-02-01 16:04:44,010] Trial 1 finished with value: 10090.323639941767 and parameters: {'max_depth': 5, 'learning_rate': 0.11352760312044687, 'n_estimators': 191, 'subsample': 0.7118736287072048, 'colsample_bytree': 0.5223808711839054, 'colsample_bylevel': 0.7329957728805684, 'colsample_bynode': 0.5459416619834728, 'reg_alpha': 0.5464547900651238, 'reg_lambda': 1.8766688715592286, 'gamma': 2.557407735508843}. Best is trial 0 with value: 6647.616758420024.
[I 2025-02-01 16:04:44,184] Trial 2 finished with value: 11115.544152050956 and parameters: {'max_depth': 4, 'learning_rate': 0.09812714606364059, 'n_estimators': 168, 'subsample': 0.8864679065756361, 'colsample_bytree': 0.5101851451534378, 'colsample_bylevel': 0.570762541594945, 'colsample_bynode': 0.5285500743058988, 'reg_alpha': 0.9727868683539994, 'reg_lambda': 3.796711620878259, 'gamma': 0.16204543037208086}. Best is trial 0 with value: 6647.616758420024.


Trial 1: RMSE 10090.323639941767
Trial 2: RMSE 11115.544152050956


[I 2025-02-01 16:04:44,395] Trial 3 finished with value: 8483.062383715784 and parameters: {'max_depth': 4, 'learning_rate': 0.09193822504499102, 'n_estimators': 186, 'subsample': 0.8960445958132534, 'colsample_bytree': 0.7867895712362286, 'colsample_bylevel': 0.5165900190693073, 'colsample_bynode': 0.6883690250736331, 'reg_alpha': 0.058428351136986495, 'reg_lambda': 2.221187586957993, 'gamma': 4.758448108125366}. Best is trial 0 with value: 6647.616758420024.


...

Trial 22: RMSE 3813.4799982509703


[I 2025-02-01 16:04:53,581] Trial 23 finished with value: 4062.872532642464 and parameters: {'max_depth': 8, 'learning_rate': 0.11023243426817066, 'n_estimators': 492, 'subsample': 0.791356882742024, 'colsample_bytree': 0.7569172920049512, 'colsample_bylevel': 0.7119009795662778, 'colsample_bynode': 0.7134560525266905, 'reg_alpha': 0.24127511846797084, 'reg_lambda': 0.18481147974599646, 'gamma': 3.015061842498286}. Best is trial 21 with value: 3743.020209262959.


Trial 23: RMSE 4062.872532642464


[I 2025-02-01 16:04:54,274] Trial 24 finished with value: 3890.430278112535 and parameters: {'max_depth': 8, 'learning_rate': 0.1501249169937477, 'n_estimators': 458, 'subsample': 0.8358338748238252, 'colsample_bytree': 0.7608304316510353, 'colsample_bylevel': 0.7632557110291239, 'colsample_bynode': 0.6721127058455731, 'reg_alpha': 0.3361185178933097, 'reg_lambda': 0.17966988289787778, 'gamma': 2.2365954510097024}. Best is trial 21 with value: 3743.020209262959.


Trial 24: RMSE 3890.430278112535


Saving the top performing hypermeters to train on the scoring dataset


In [95]:
import json 
top_trials = sorted(study.trials, key=lambda x: x.value)[:3]
top_params = [trial.params for trial in top_trials]

# Save the top parameters to a JSON file
with open("best_models.json", "w") as f:
    json.dump(top_params, f, indent=4)

In [96]:
with open("best_models.json", "r") as f:
    best_params = json.load(f)

# Load the new dataset without index
scoring_data = pd.read_csv("scoring_cleaned_data.csv")

# Define categorical columns (must match those from training)
categorical_cols = ["Vehicle Category", "Fuel Type", "Fuel Technology", "Electric Mile Range"]

In [98]:
# Ensure columns exist in scoring data
missing_cols = [col for col in categorical_cols if col not in scoring_data.columns]
if missing_cols:
    raise ValueError(f"Missing categorical columns in scoring dataset: {missing_cols}")

# Separate features and target variable
X_scoring = scoring_data.drop(columns=["Vehicle Population"], errors="ignore")  # Adjust column name if needed
y_scoring = scoring_data["Vehicle Population"] if "Vehicle Population" in scoring_data else None  # Handle missing target

# Encode categorical features using the same method as training
encoder = TargetEncoder(cols=categorical_cols)
X_train_encoded = X_train.copy()
X_train_encoded[categorical_cols] = encoder.fit_transform(X_train[categorical_cols], y_train)

scoring_encoded = X_scoring.copy()
scoring_encoded[categorical_cols] = encoder.transform(X_scoring[categorical_cols])

In [100]:
# Ensure numerical columns are correctly formatted
numerical_cols = ["GVWR Class", "Number of Vehicles Registered at the Same Address"]
for col in numerical_cols:
    if col in scoring_encoded.columns:
        scoring_encoded[col] = scoring_encoded[col].astype("category").cat.codes

In [101]:
# Dictionary to store RMSE values
rmse_scores = {}
predictions = {}

# Apply the best models to make predictions and compute RMSE
for i, params in enumerate(best_params):
    model = XGBRegressor(**params)
    
    # Train model on original dataset
    model.fit(X_train_encoded, y_train)
    
    # Predict on scoring dataset
    y_pred = model.predict(scoring_encoded)
    predictions[f"Model_{i+1}"] = y_pred
    
    # Compute RMSE (only if y_scoring exists)
    if y_scoring is not None:
        rmse = np.sqrt(mean_squared_error(y_scoring, y_pred))
        rmse_scores[f"Model_{i+1}"] = rmse
        print(f"Model {i+1} RMSE: {rmse}")

Model 1 RMSE: 13128.496029629594
Model 2 RMSE: 16270.675953997732
Model 3 RMSE: 16570.272176400726
